# 🔪 Resurface Fine-Tuning (Fixed Version)

Fine-tunes TinyLlama 1.1B on vulnerability playbooks.

**Baseline:** 25.9% (Llama 3.3 70B without training)

**Instructions:**
1. Runtime → Change runtime type → GPU (T4)
2. Runtime → Run all
3. Wait ~20-30 minutes

In [ ]:
# Cell 1: Install dependencies (pinned for Colab Feb 2026)
!pip uninstall -y -q transformers
!pip install -q transformers==4.40.0 datasets peft accelerate bitsandbytes==0.43.0 trl==0.9.6 huggingface_hub
print("✅ Done! Now RESTART RUNTIME (Runtime → Restart runtime) then run from Cell 2")

In [ ]:
# Cell 2: Check GPU
import torch
assert torch.cuda.is_available(), "No GPU! Go to Runtime → Change runtime type → GPU"
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Cell 3: Download training data
!wget -q https://raw.githubusercontent.com/Ibrahim-sec/resurface/master/training/data/train.jsonl
!wc -l train.jsonl

In [ ]:
# Cell 4: Load and format data
import json
from datasets import Dataset

examples = []
with open("train.jsonl") as f:
    for line in f:
        data = json.loads(line)
        # Simple format
        text = f"### Human: {data['messages'][0]['content']}\n\n### Assistant: {data['messages'][1]['content']}"
        examples.append({"text": text})

dataset = Dataset.from_list(examples)
print(f"Loaded {len(dataset)} examples")

In [ ]:
# Cell 5: Load model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

print(f"Loading {MODEL}...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.pad_token = tokenizer.eos_token
print("Model loaded!")

In [ ]:
# Cell 6: Setup LoRA
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
# Cell 7: Train
from trl import SFTTrainer
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=args,
)

print("Starting training...")
trainer.train()
print("Training complete!")

In [ ]:
# Cell 8: Save to Drive
from google.colab import drive
drive.mount('/content/drive')

SAVE_PATH = "/content/drive/MyDrive/resurface-lora"
model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
print(f"Saved to {SAVE_PATH}")

In [ ]:
# Cell 9: Test it!
prompt = """### Human: You are a security researcher. Vulnerability Type: sqli. Scenario: A search feature that queries a database directly. Generate exploitation steps.

### Assistant:"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=500, temperature=0.7, do_sample=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))